## Машинное обучение в бизнесе

### Урок 9. Кейс 2. Внедрение модели в продукцию

Создать веб-сервис на Flask через Google Colab или на локальной машине. Научиться отправлять post-запросы с помощью Postman и получать на них ответы. Можно использовать любую из обученных моделей в заданиях ко 2 и 3 занятиям.

In [1]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [2]:
# # Пробный запуск Flask

# app = Flask(__name__)
# run_with_ngrok(app)  # Start ngrok when app is run

# @app.route("/a")
# def hello():
#     return "Hello World!"

# if __name__ == '__main__':
#     app.run()

In [3]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.241-b07, mixed mode)


C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\server.py:379: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"


  Starting server from C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\rajul\AppData\Local\Temp\tmp6af8ozm6
  JVM stdout: C:\Users\rajul\AppData\Local\Temp\tmp6af8ozm6\h2o_rajul_started_from_python.out
  JVM stderr: C:\Users\rajul\AppData\Local\Temp\tmp6af8ozm6\h2o_rajul_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,06 secs
H2O cluster timezone:,Europe/Moscow
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,"14 days, 1 hour and 5 minutes"
H2O cluster name:,H2O_from_python_rajul_gf7rry
H2O cluster total nodes:,1
H2O cluster free memory:,247.5 Mb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


In [4]:
# Загружаем обученные модели

model_glm_poisson = h2o.load_model('GLM_model_python_1581997959683_20')
model_glm_gamma = h2o.load_model('GLM_model_python_1581997959683_21')

In [ ]:
# Обработчики и запуск Flask

def map_for_dict_Gender(Gender):
  dict_Gender = {'Male':0, 'Female':1}
  res = dict_Gender.get(Gender)
  return res

def map_for_dict_MariStat(MariStat):
  dict_MariStat = {'Other':0, 'Alone':1}
  res = dict_MariStat.get(MariStat)
  return res

def f_VehUsage_Professional(VehUsage):
  if VehUsage == 'Professional':
    VehUsage_Professional = 1
  else:
    VehUsage_Professional = 0
  return(VehUsage_Professional)

def f_VehUsage_Private_trip_to_office(VehUsage):
  if VehUsage == 'Private+trip to office':
    VehUsage_Private_trip_to_office = 1
  else:
    VehUsage_Private_trip_to_office = 0
  return(VehUsage_Private_trip_to_office)

def f_VehUsage_Private(VehUsage):
  if VehUsage == 'Private':
    VehUsage_Private = 1
  else:
    VehUsage_Privatel = 0
  return(VehUsage_Private)

def f_VehUsage_Professional_run(VehUsage):
  if VehUsage == 'Professional run':
    VehUsage_Professional_run = 1
  else:
    VehUsage_Professional_run = 0
  return(VehUsage_Professional_run)

def return_NewH2o_Frame():
  columns = [
      'LicAge',
      'Gender',
      'MariStat',
      'DrivAge',
      'HasKmLimit',
      'BonusMalus',
      'OutUseNb',
      'RiskArea',
      'VehUsg_Private',
      'VehUsg_Private+trip to office',
      'VehUsg_Professional',
      'VehUsg_Professional run',
      'CSP1',
      'CSP2',
      'CSP3',
      'CSP6',
      'CSP7',
      'CSP20',
      'CSP21',
      'CSP22',
      'CSP26',
      'CSP37',
      'CSP40',
      'CSP42',
      'CSP46',
      'CSP47',
      'CSP48',
      'CSP49',
      'CSP50',
      'CSP55',
      'CSP56',
      'CSP57',
      'CSP60',
      'CSP65',
      'CSP66'
      ]
  return h2o.H2OFrame([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]], column_names = columns)

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/predict', methods=['GET', 'POST'])
def predict3():

    try:

      json_input = request.json

      ID = json_input["ID"]
      LicAge = json_input["LicAge"]
      Gender = map_for_dict_Gender(json_input["Gender"])
      MariStat = map_for_dict_MariStat(json_input["MariStat"])
      DrivAge = json_input["DrivAge"]
      HasKmLimit = json_input["HasKmLimit"]
      BonusMalus = json_input["BonusMalus"]
      OutUseNb = json_input["OutUseNb"]
      RiskArea = json_input["RiskArea"]
      VehUsg_Private = f_VehUsage_Private(json_input["VehUsage"])
      VehUsg_Private_trip_to_office = f_VehUsage_Private_trip_to_office(json_input["VehUsage"])
      VehUsg_Professional = f_VehUsage_Professional(json_input["VehUsage"])
      VehUsg_Professional_run = f_VehUsage_Professional_run(json_input["VehUsage"])
      CSP1 = 0
      CSP2 = 0
      CSP3 = 0
      CSP6 = 0
      CSP7 = 0
      CSP20 = 0
      CSP21 = 0
      CSP22 = 0
      CSP26 = 0
      CSP37 = 0
      CSP40 = 0
      CSP42 = 0
      CSP46 = 0
      CSP47 = 0
      CSP48 = 0
      CSP49 = 0
      CSP50 = 0
      CSP55 = 0
      CSP56 = 0
      CSP57 = 0
      CSP60 = 0
      CSP65 = 0
      CSP66 = 0

      hf = return_NewH2o_Frame()

      hf[0, 'LicAge'] = LicAge
      hf[0, 'Gender'] = Gender
      hf[0, 'MariStat'] = MariStat
      hf[0, 'DrivAge'] = DrivAge
      hf[0, 'HasKmLimit'] = HasKmLimit
      hf[0, 'BonusMalus'] = BonusMalus
      hf[0, 'OutUseNb'] = OutUseNb
      hf[0, 'RiskArea'] = RiskArea
      hf[0, 'VehUsg_Private'] = VehUsg_Private
      hf[0, 'VehUsg_Private+trip to office'] = VehUsg_Private_trip_to_office
      hf[0, 'VehUsg_Professional'] = VehUsg_Professional
      hf[0, 'VehUsg_Professional run'] = VehUsg_Professional_run
      hf[0, 'CSP1'] = CSP1
      hf[0, 'CSP2'] = CSP2
      hf[0, 'CSP3'] = CSP3
      hf[0, 'CSP6'] = CSP6
      hf[0, 'CSP7'] = CSP7
      hf[0, 'CSP20'] = CSP20
      hf[0, 'CSP21'] = CSP21
      hf[0, 'CSP22'] = CSP22
      hf[0, 'CSP26'] = CSP26
      hf[0, 'CSP37'] = CSP37
      hf[0, 'CSP40'] = CSP40
      hf[0, 'CSP42'] = CSP42
      hf[0, 'CSP46'] = CSP46
      hf[0, 'CSP47'] = CSP47
      hf[0, 'CSP48'] = CSP48
      hf[0, 'CSP49'] = CSP49
      hf[0, 'CSP50'] = CSP50
      hf[0, 'CSP55'] = CSP55
      hf[0, 'CSP56'] = CSP56
      hf[0, 'CSP57'] = CSP57
      hf[0, 'CSP60'] = CSP60
      hf[0, 'CSP65'] = CSP65
      hf[0, 'CSP66'] = CSP66

      prediction_Poisson = model_glm_poisson.predict(hf)
      value_Poisson  = prediction_Poisson.as_data_frame()['predict'][0]
      prediction_Gamma = model_glm_gamma.predict(hf)
      value_Gamma  = prediction_Gamma.as_data_frame()['predict'][0]
      value_BurningCost = value_Poisson * value_Gamma

      return jsonify({'ID':ID, 'value_Poisson':value_Poisson, 'value_Gamma':value_Gamma, 'value_BurningCost':value_BurningCost}) 
    
    except:
      
      return "Error"



if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://4c6bce7c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Parse progress: |█████████████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%




response:
{"ID":1,"value_BurningCost":110.82390163199808,"value_Gamma":431.58001180103514,"value_Poisson":0.25678645581735043}